In [1]:
# Xavier Initialization
# Non Saturating Activation Functions
# Gradient clipping
# Early Stopping
# L1 and L2 Regularization

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
from datetime import datetime

In [3]:
from tensorflow.contrib.layers import fully_connected

In [4]:
n_inputs = 28 * 28 
n_hidden_1 = 300
n_hidden_2 = 100
n_outputs = 10

In [5]:
# Construction phase
X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(dtype=tf.int64, shape=(None), name='y')

# Variance Scaling initializer
he_init = tf.contrib.layers.variance_scaling_initializer()
hidden_1 = fully_connected(X, n_hidden_1, weights_initializer=he_init, scope='hidden_1')

# Xavier Initialization - Default weight initialization
hidden_2 = fully_connected(hidden_1, n_hidden_2, scope='hidden_2')
logits = fully_connected(hidden_2, n_outputs, activation_fn=None, scope='hidden_3')

with tf.name_scope('loss'):
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='x_entropy')
    loss = tf.reduce_mean(x_entropy, name ='loss')

learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [6]:

now = datetime.utcnow().strftime("%Y-%m-%d-%H:%M:%S")
root_logdir = "tf_logs"
logdir = os.path.join(os.getcwd(),"{}/run-{}".format(root_logdir, now))

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [7]:
n_epochs = 10
batch_size = 50
mnist = input_data.read_data_sets("tmp/data")

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
accuracy_summary = tf.summary.scalar('Accuracy', accuracy)

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

sess = tf.InteractiveSession()
init.run()

for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    file_writer.add_summary(accuracy_summary.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels}),epoch)
    
    print(epoch, "Train accuracy ", acc_train, "Test accuracy ", acc_test)

save_path = saver.save(sess, os.path.join(os.getcwd(),"Models/Initialization_Model.ckpt"))

0 Train accuracy  0.88 Test accuracy  0.902
1 Train accuracy  0.94 Test accuracy  0.9217
2 Train accuracy  0.9 Test accuracy  0.9343
3 Train accuracy  0.92 Test accuracy  0.9405
4 Train accuracy  0.96 Test accuracy  0.9449
5 Train accuracy  0.96 Test accuracy  0.9491
6 Train accuracy  0.98 Test accuracy  0.9515
7 Train accuracy  1.0 Test accuracy  0.9546
8 Train accuracy  0.96 Test accuracy  0.9562
9 Train accuracy  1.0 Test accuracy  0.9589


In [10]:
sess.close()
file_writer.close()